<a href="https://colab.research.google.com/github/movie112/INU-DILAB/blob/main/210929_211005/paper_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
<https://arxiv.org/abs/1810.04805>
---


### BERT: Bidirectional Encoder Representations from Transformer
- transformer의 encoder만을 bidirection하게 사용하겠다.
- Pre-traning
  1. Masked language model(MLM): 특정한 단어 representation을 학습하기 위해 해당 단어의 선행, 후행 단어 모두 사용. 일정 부분 masking해놓고
  2. Next sentence prediction(NSP): 입력 두 sentence, 다음 sentence가 실제로 corpus에서 다음에 등장하는지 학습
- Fine-tuning
  - BERT 윗 단에 하나의 layer를 쌓음
  


### model architecture
- multi-layer bidirectional trasformer encoder
  - L: number of layers (Transformer block)
  - H: hidden size
  - A: number of self attention heads

<img src="https://cdn-images-1.medium.com/max/1600/1*9W5_CpuM3Iq09kOYyK9CeA.png" width="120px" height="250px"></img>

- BERT(base)
  - L=12, H=768, A=12
  - total parameters = 110M
  - same model size as OpenAI GPT
- BERT(large)
  - L=24, H=1024, A=16
  - total parameters = 340M



### input/output representations
- 다양한 down-stream tasks를 다루기 위해 유연하게 구성될 필요
  - a single sentence / a pair of sentences
    - sentence: an arbitrary span of contiguous text, rather than an actual linguistic sentence(완성된 문장이 아닌 단순히 연속된 단어들의 나열)
    - sequence: the input token sequences to BERT, which may be a single sentence or two sentences packed together(single sentence 일수도 두 개의 sentence가 결합된 형태일수도)

<img src="https://pytorch.org/tutorials/_images/bert.png" width="300px" height="250px"></img>

- 입력으로 maked sentence가 들어간다.
- [CLS]: the first token of every sequence
- [SEP]: special token separating two sentences
- C: the final hidden vector of special [CLS] token
  - BERT layer 끝, 단순한 binary classification에 쓰임(NSP)
- 각각의 T: the final hidden vector of N번째 token(MLM)

- **input representation**
<img src="https://miro.medium.com/max/619/1*iJqlhZz-g6ZQJ53-rE9VvA.png" width="500px" height="150px"></img>

- 세 가지 embedding vector의 합으로 구성
  1. Token embedding: WordPiece embeddings 일반적으로 30,000 token
  2. Segment embedding: sequence 두 개 들어갔을 때, Ea, Eb식으로 구분
  3. Position embedding: 각각의 position에 대한 정보


## per-training BERT
- Task1: masked language model(MLM)
  - 전체 sequence의 15%가 랜덤하게 mask token으로
  <img src="https://cdn-images-1.medium.com/max/2000/0*ViwaI3Vvbnd-CJSQ.png" width="300px" height="150px"></img>
  - mask된 W4 -> BERT -> output4 -> classification layer -> W'4
  - W'4 = W4가 되도록 학습시켜야 함
  - 문제점
    - masking이 pre-training 과정에서만 일어나고 fine-tuning 에서는 일어나지 않음 -> pre-trainng과 fine-tuning에서 mask token의 missmatch가 일어날 수 있음
  - 해결
    - i번 째 token이 masking 될 것이라고 선택 된다면, 그 중 80%만 실제로 masking, 10%는 랜덤한 token으로 치환, 10%는 unchange
    - 경험에 대한 해결법
- Task2: next sentence prediction(NSP)
  - 다음 문장 예측
  - QA, NLI는 기본적으로 두 문장 이상의 관계를 해석해야 풀 수 있는데, 문장 단위의 언어모델은 잘 못한다.
  - BERT는 Binarized next sentence prediction 제시
    - 50%는 A, B가 실제로 이어진 문장에서 데이터를 뽑아옴(IsNext)
    - 50%는 실제로 다음 문장이 아닌 것으로 구성(NotNext)
    - C is used for next sentence prediction
    - 이를 pre-training하는 것이 도움이 많이 됨

  

### differences in pre-training model architectures
<img src="https://imgs.developpaper.com/imgs/342513798-5e52113a0bfb6_articlex.png" width="500px" height="150px"></img>


- pre-train 시, max sequence token length = 128, 90%의 step
  - 10%는 512

## Fine-tuning BERT
<img src="https://nlpinkorean.github.io/images/bert/bert-tasks.png" width="500px" height="300px"></img>
- BERT 위에 layer 하나 쌓기
- a) 한 sequence가 두 sentence를 받아 class에 속하는지 아닌지 
- b) 1 sentence 받아 감정분석 같은 label
- c) question과 paragraph 를 넣어서 답 생성
- d) 1 sentence 각각의 token마다 답 생성(개체명인식, 형태소 분석)



## Experiments
- effect of pre-training task
  - BERT(base)와 No NSP를 비교하였을 때, 두 sentence를 필요로 하는 task는 유의미한 성능 보임
    - NSP pre-training의 필요성
- effect of model size
  - 구조가 커질수록 성능 좋음(당연)
- feature-based approach with BERT
  - fine-tuning 거치지 않고 pre-training된 모델만 가지고 한다면?
    - 성능을 비교하면 마지막 4개 layers의 정보를 사용했을 때 fine-tuning한 것 만큼의 성능을 보임